#### A CNN model is created and trained on the char54k dataset to identify digits 1 to 9

## Function to calculate the COM shift

In [23]:
import os
import numpy
from scipy import ndimage
import cv2

This function is used to centralize the image using its center of mass

In [24]:
def getBestShift(img):
    cy,cx = ndimage.measurements.center_of_mass(img)
    rows,cols = img.shape
    shiftx = numpy.round(cols/2.0-cx).astype(int)
    shifty = numpy.round(rows/2.0-cy).astype(int)
    return shiftx,shifty

## Function to create training data

This function is used to read image traning data from the folders named 1, 2,..., 9 in the DigitImages directory

In [25]:
def createTrainingData():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
            new_array = cv2.resize(img_array, (img_rows, img_cols))
            new_array = shiftCenterOfMass(new_array)
            training_data.append([new_array, class_num])

## Function to centralize image

This function will centralize the image according to its center of mass.Same has been implemented in sudokuSolver.
Without this step, the accuracy drops from nearly 100% to 70-80%

In [26]:
def shiftCenterOfMass(img):
    img = cv2.bitwise_not(img)

    # Centralize the image according to center of mass
    shiftx,shifty = getBestShift(img)
    shifted = shift(img,shiftx,shifty)
    img = shifted

    img = cv2.bitwise_not(img)
    return img

## Function to shift image

This function is used to centralize the image according to its center of mass

In [27]:
def shift(img,sx,sy):
    rows,cols = img.shape
    M = numpy.float32([[1,0,sx],[0,1,sy]])
    shifted = cv2.warpAffine(img,M,(cols,rows))
    return shifted

## Import Required Libraries

In [28]:
#Neural Network modules
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical


import numpy                    #Module for working with arrays
import os                       #Standard module for interacting with OS
import random                   #For shuffling images
import cv2                      #Standard module in solving computer visions
from scipy import ndimage       #Library for multidimensional image processing

## Create and build CNN

In [29]:
batch_size = 128
num_classes = 9
epochs = 50

# input image dimensions
img_rows, img_cols = 28, 28

DATADIR = "digits_jpeg"
CATEGORIES = ["1","2","3","4","5","6","7","8","9"]

training_data = []

createTrainingData()

# Mix data up
random.shuffle(training_data)

# Split 80-20
x_train = []
y_train = []
x_test = []
y_test = []
for i in range(len(training_data)*8//10):
    x_train.append(training_data[i][0])
    y_train.append(training_data[i][1])
for i in range(len(training_data)*8//10,len(training_data)):
    x_test.append(training_data[i][0])
    y_test.append(training_data[i][1])

# Reshape
x_train = numpy.array(x_train)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = numpy.array(x_test)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize the data
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer="Adam",metrics=['accuracy'])

model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

model.save('digitRecognition22.keras')

C:\Users\Shreyas\AppData\Local\Temp\ipykernel_7028\1076359088.py:2: DeprecationWarning: Please import `center_of_mass` from the `scipy.ndimage` namespace; the `scipy.ndimage.measurements` namespace is deprecated and will be removed in SciPy 2.0.0.
  cy,cx = ndimage.measurements.center_of_mass(img)


Epoch 1/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 9s 113ms/step - accuracy: 0.5940 - loss: 1.2247 - val_accuracy: 0.9476 - val_loss: 0.1802
Epoch 2/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 10s 99ms/step - accuracy: 0.9428 - loss: 0.2011 - val_accuracy: 0.9754 - val_loss: 0.0968
Epoch 3/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 6s 101ms/step - accuracy: 0.9618 - loss: 0.1219 - val_accuracy: 0.9798 - val_loss: 0.0672
Epoch 4/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 6s 100ms/step - accuracy: 0.9749 - loss: 0.0753 - val_accuracy: 0.9853 - val_loss: 0.0567
Epoch 5/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 10s 100ms/step - accuracy: 0.9831 - loss: 0.0557 - val_accuracy: 0.9853 - val_loss: 0.0434
Epoch 6/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 6s 102ms/step - accuracy: 0.9817 - loss: 0.0551 - val_accuracy: 0.9902 - val_loss: 0.0318
Epoch 7/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 10s 101ms/step - accuracy: 0.9889 - loss: 0.0366 - val_accuracy: 0.9929 - val_loss: 0.0271
Epoch 8/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 6s 102ms/step - accuracy: 0.9884 - loss: 0.0292 - val_accuracy: 